## Part 1 : Get the Date<br>

In [1]:
import pandas as pd

In [2]:
import praw
reddit = praw.Reddit(client_id='8G3apa5U51Rw2g',
                     client_secret='ACPdbSJSDLV8U3EAjla2MhOj1zg',
                     user_agent='my user agent')
print(reddit.read_only) 

True


#### Get Top 1000 posts from r/UpliftingNews, r/HumansBeingBros, r/NoSleep, and r/Creepy

In [3]:
l = 1000
again = False

In [4]:
if again:
    td_neg_1 = pd.DataFrame(columns=['Text'])
    for submission in reddit.subreddit('upliftingnews').top('year',limit=l):
        td_neg_1 = td_neg_1.append({'Text': submission.title}, ignore_index=True)
    td_neg_1.insert(0, 'Sentiment', 1)
    td_neg_1.columns = ["Sentiment","Text"]

In [5]:
if again:
    td_neg_2 = pd.DataFrame(columns=['Text'])
    for submission in reddit.subreddit('humansbeingbros').top('year',limit=l):
        td_neg_2 = td_neg_2.append({'Text': submission.title}, ignore_index=True)
    td_neg_2.insert(0, 'Sentiment', 1)
    td_neg_2.columns = ["Sentiment","Text"]


In [6]:
if again:
    td_pos_1 = pd.DataFrame(columns=['Text'])
    for submission in reddit.subreddit('nosleep').top('year',limit=l):
        td_pos_1 = td_pos_1.append({'Text': submission.title}, ignore_index=True)
    td_pos_1.insert(0, 'Sentiment', 0)
    td_pos_1.columns = ["Sentiment","Text"]


In [7]:
if again:
    td_pos_2 = pd.DataFrame(columns=['Text'])
    for submission in reddit.subreddit('creepy').top('year',limit=l):
        td_pos_2 = td_pos_2.append({'Text': submission.title}, ignore_index=True)
    td_pos_2.insert(0, 'Sentiment', 0)
    td_pos_2.columns = ["Sentiment","Text"]


#### Save the data

In [8]:
if again:
    train_data_df = pd.concat([td_pos_1,td_pos_2,td_neg_1,td_neg_2])
    train_data_df.to_pickle("./savedData.pkl")
    #test_data_df.to_pickle("./saveTest.pkl")

## Part 2 : Clean the Data<br>

In [9]:
train_data_df = pd.read_pickle("./savedData.pkl")
train_data_df.head()
#test_data_df = pd.read_pickle("./saveTest.pkl")
#test_data_df.head()

,Sentiment,Text
0,0,She Sold Happiness in Glass Jars
1,0,"If you can see this, it is very important that..."
2,0,My girlfriend talks in her sleep. She's been s...
3,0,Something went wrong with my heart transplant
4,0,I Answered a Spam Call


#### Remove irrelevant characters, lowercase, tokenize, stemming

In [10]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
def tokenizeMe(text):
    text = re.sub("[^a-zA-Z]", " ", lemmatizer.lemmatize(text))
    tokens = [t for t in nltk.word_tokenize(text) if re.search('[a-zA-Z]', t)]
    lemma = [item for item in tokens if len(item)>2]
    return(lemma)

##### Convert each post to a Vector of token counts<br>
##### tri-grams<br>
##### top 200 features

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range = (0,3)
                             ,analyzer = 'word'
                             ,tokenizer = tokenizeMe
                             ,lowercase = True
                             ,stop_words = 'english'
                             ,max_features = 100)
features = vectorizer.fit_transform(train_data_df.Text.tolist())
features_nd = features.toarray()

#### Split into Training 90% and Testing Data 10%

In [12]:
from sklearn.model_selection  import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(features_nd[0:len(train_data_df)], train_data_df.Sentiment,test_size=0.1, random_state=1)

## Part 3 : Use the Data<br>

#### Perform Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(solver='lbfgs',max_iter=1000)
X=features_nd[0:len(train_data_df)]
y=train_data_df.Sentiment
log_model = log_model.fit(X, y)

## Part 4 : Analyse the Results<br>

In [14]:
y_pred = log_model.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.88      0.85       527
           1       0.87      0.81      0.84       549

   micro avg       0.84      0.84      0.84      1076
   macro avg       0.85      0.85      0.84      1076
weighted avg       0.85      0.84      0.84      1076



In [15]:
print(vectorizer.get_feature_names())

['', 'ago', 'away', 'baby', 'best', 'birthday', 'boy', 'bro', 'bros', 'brother', 'bus', 'cancer', 'car', 'cat', 'children', 'city', 'dad', 'daughter', 'day', 'days', 'dead', 'death', 'dog', 'dogs', 'don', 'donates', 'driver', 'family', 'food', 'free', 'friend', 'game', 'gets', 'girl', 'giving', 'going', 'good', 'got', 'guy', 'help', 'helping', 'helps', 'high', 'home', 'homeless', 'hospital', 'house', 'human', 'job', 'just', 'kid', 'kids', 'know', 'left', 'life', 'like', 'little', 'local', 'love', 'make', 'man', 'million', 'mom', 'mother', 'need', 'new', 'night', 'old', 'parents', 'people', 'plastic', 'police', 'post', 'rescue', 'room', 'save', 'saved', 'saves', 'saving', 'saw', 'school', 'son', 'state', 'stop', 'stuck', 'students', 'think', 'time', 'today', 'town', 'water', 'went', 'wife', 'woman', 'work', 'world', 'year', 'year old', 'years', 'young']


In [16]:
a = vectorizer.get_feature_names() 
b = log_model.coef_[0]
c = list(zip(a,b))

In [17]:
c.sort(key = lambda x: x[1]) 
print('\n'.join(map(str,c)))

('think', -2.373349394634655)
('dead', -2.2291746793884597)
('night', -2.2271903685205863)
('room', -2.1581187747865593)
('death', -1.9410130718950933)
('went', -1.6288572558335315)
('brother', -1.205428257723142)
('saw', -1.2021681586080415)
('don', -1.1918456498896535)
('house', -1.168850393458495)
('going', -0.9920375421442347)
('ago', -0.9748843176603189)
('son', -0.963780545963733)
('town', -0.9002569318613411)
('work', -0.8699270425338524)
('job', -0.8646266274411799)
('friend', -0.8585100866996358)
('know', -0.8459260353749647)
('wife', -0.8111375745611389)
('daughter', -0.7446471373936007)
('parents', -0.7046059539610284)
('like', -0.6947121623126306)
('old', -0.6931703307227054)
('just', -0.6562737726267913)
('life', -0.6527945430820428)
('left', -0.5916087236529475)
('stuck', -0.5839670345575412)
('got', -0.571220713439041)
('away', -0.4281460601232218)
('days', -0.40327242051468964)
('human', -0.3741977597475917)
('today', -0.36852483562980076)
('game', -0.31037447822132036)